In [ ]:
import os
import numpy as np
import librosa
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report, accuracy_score
from geopy.distance import geodesic
from google.colab import files
import IPython.display as ipd
import random

# Function to extract features (Mel-spectrogram, chroma, spectral contrast)
def extract_features(file_path):
    try:
        audio, sample_rate = librosa.load(file_path, sr=None)

        # Mel-spectrogram
        mel_spec = librosa.feature.melspectrogram(y=audio, sr=sample_rate, n_mels=128)
        mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)

        # Chroma features
        chroma = librosa.feature.chroma_stft(y=audio, sr=sample_rate)

        # Spectral contrast
        spectral_contrast = librosa.feature.spectral_contrast(y=audio, sr=sample_rate)

        # Combine features
        features = np.concatenate((
            np.mean(mel_spec_db, axis=1),
            np.mean(chroma, axis=1),
            np.mean(spectral_contrast, axis=1)
        ))
        return features
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None

# Build a CNN model
def build_cnn(input_shape, num_classes):
    model = Sequential([
        Dense(256, activation='relu', input_shape=input_shape),
        Dropout(0.3),
        Dense(128, activation='relu'),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Function to simulate traffic signal control
def control_traffic_signal(ambulance_direction):
    if ambulance_direction == 'North':
        print("Changing signal to GREEN in the North direction.")
    elif ambulance_direction == 'East':
        print("Changing signal to GREEN in the East direction.")
    elif ambulance_direction == 'South':
        print("Changing signal to GREEN in the South direction.")
    elif ambulance_direction == 'West':
        print("Changing signal to GREEN in the West direction.")
    else:
        print("Unknown direction. No signal change.")

# Function to simulate GPS location of an ambulance
def get_ambulance_location():
    lat = random.uniform(28.0, 29.0)
    lon = random.uniform(77.0, 78.0)
    return lat, lon

# Function to simulate RFID-based authentication for an ambulance
def authenticate_ambulance(rfid_tag):
    authorized_rfid_tags = ['AM123', 'AM456', 'AM789']
    return rfid_tag in authorized_rfid_tags

# Function to simulate ambulance direction based on GPS location
def get_ambulance_direction(start_lat, start_lon, end_lat, end_lon):
    start = (start_lat, start_lon)
    end = (end_lat, end_lon)
    distance = geodesic(start, end).km
    direction = 'North' if start_lat < end_lat else 'South'
    return direction, distance

# Train the model
def main():
    print("Please upload your own audio files to train the model:")

    uploaded = files.upload()
    features = []
    labels = []

    for file_name in uploaded.keys():
        print(f"Processing {file_name}...")

        # Prompt user for label (siren or non-siren)
        label = input(f"Enter label for {file_name} ('siren' or 'non-siren'): ")

        # Extract features
        feature = extract_features(file_name)
        if feature is not None:
            features.append(feature)
            labels.append(label)

    X = np.array(features)
    y = np.array(labels)

    if len(X) < 2:
        print("Insufficient data for training. Please upload more audio files.")
        return None, None

    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    y_categorical = to_categorical(y_encoded)

    test_size = 0.2 if len(X) > 4 else 0.5
    X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=test_size, random_state=42)

    input_shape = (X_train.shape[1],)
    num_classes = y_categorical.shape[1]
    model = build_cnn(input_shape, num_classes)

    print("Training model...")
    history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test), verbose=1)

    y_pred = model.predict(X_test)
    y_pred_labels = np.argmax(y_pred, axis=1)
    y_test_labels = np.argmax(y_test, axis=1)

    print("\nClassification Report:")
    print(classification_report(y_test_labels, y_pred_labels, target_names=label_encoder.classes_))

    accuracy = accuracy_score(y_test_labels, y_pred_labels)
    print(f"Accuracy: {accuracy:.2f}")

    return model, label_encoder

# Function to upload and classify a new sound
def classify_sound(model, label_encoder):
    print("\nPlease upload a sound file for classification:")
    uploaded = files.upload()

    for file_name in uploaded.keys():
        print(f"Processing {file_name}...")
        ipd.display(ipd.Audio(file_name))

        feature = extract_features(file_name)
        if feature is not None:
            prediction = model.predict(np.array([feature]))
            predicted_label = label_encoder.classes_[np.argmax(prediction)]
            print(f"Predicted Label: {predicted_label}")
        else:
            print("Error extracting features.")

# Simulate ambulance detection and controlling traffic signal
def simulate_ambulance_and_control_signal():
    print("\nSimulating ambulance detection and controlling traffic signal:")

    start_lat, start_lon = get_ambulance_location()
    end_lat, end_lon = get_ambulance_location()
    direction, distance = get_ambulance_direction(start_lat, start_lon, end_lat, end_lon)
    print(f"Ambulance detected. Direction: {direction}, Distance: {distance:.2f} km")

    rfid_tag = input("Enter RFID tag of the ambulance: ")
    if authenticate_ambulance(rfid_tag):
        print("RFID authentication successful.")
        control_traffic_signal(direction)
    else:
        print("RFID authentication failed. No signal change.")

if __name__ == "__main__":
    # Train the model
    model, label_encoder = main()

    if model and label_encoder:
        # Classify new sounds
        classify_sound(model, label_encoder)

        # Simulate ambulance detection and traffic signal control
        simulate_ambulance_and_control_signal()


Please upload your own audio files to train the model:


Saving car-horn-90973.mp3 to car-horn-90973.mp3
Saving diesel-horn-02-98042.mp3 to diesel-horn-02-98042.mp3
Saving old-car-horn-153262.mp3 to old-car-horn-153262.mp3
Saving vintage-car-horn-153264.mp3 to vintage-car-horn-153264.mp3
Saving air-horn-273892.mp3 to air-horn-273892.mp3
Saving car-horn-6408.mp3 to car-horn-6408.mp3
Saving automobile-horn-153260.mp3 to automobile-horn-153260.mp3
Saving loud-train-horn-274373.mp3 to loud-train-horn-274373.mp3
Saving ambulance-in-city-traffic-139030.mp3 to ambulance-in-city-traffic-139030 (1).mp3
Saving siren-253181.mp3 to siren-253181 (1).mp3
Saving police-siren-21498.mp3 to police-siren-21498 (1).mp3
Saving police-siren-99029.mp3 to police-siren-99029 (1).mp3
Saving ambulance-6399.mp3 to ambulance-6399 (1).mp3
Saving distant-ambulance-siren-6108.mp3 to distant-ambulance-siren-6108 (2).mp3
Saving sound-effect-uk-ambulance-siren-164354.mp3 to sound-effect-uk-ambulance-siren-164354 (2).mp3
Saving siren-passing-275127.mp3 to siren-passing-275127 

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.6923 - loss: 14.7043 - val_accuracy: 0.2500 - val_loss: 29.2388
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.3846 - loss: 35.1561 - val_accuracy: 0.2500 - val_loss: 27.4665
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 0.6154 - loss: 17.7773 - val_accuracy: 0.2500 - val_loss: 13.3011
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.5385 - loss: 25.8310 - val_accuracy: 0.7500 - val_loss: 0.8658
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.6154 - loss: 6.7963 - val_accuracy: 0.7500 - val_loss: 7.4194
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.3077 - loss: 21.9047 - val_accuracy: 0.7500 - val_loss: 7.7954
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.3077 - loss: 28.4956 - val_accuracy: 0.7500 - val_loss: 4.4550
Epoch 8/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.5385 - loss: 14.0316 - val_accuracy: 0.2500 - val_loss: 4.6077
Epo

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Saving car-horn-90973.mp3 to car-horn-90973 (1).mp3
Processing car-horn-90973 (1).mp3...


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
Predicted Label: non-siren

Simulating ambulance detection and controlling traffic signal:
Ambulance detected. Direction: South, Distance: 39.81 km
Enter RFID tag of the ambulance: fff
RFID authentication failed. No signal change.
